<a href="https://colab.research.google.com/github/igorcduarte/Agente_PC_Builder/blob/main/Assistente_de_Montagem_de_PCs_com_IA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip -q install google-genai

In [ ]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [ ]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [ ]:
# Pergunta ao Gemini uma informação mais recente que seu conhecimento

from IPython.display import HTML, Markdown

# Perguntar pro modelo quando uma informação recente
resposta = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é o próximo Google I/O?',
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {resposta.text}"))

Resposta:
 O próximo Google I/O será nos dias **14 e 15 de maio de 2024**.


In [ ]:
# Pergunta ao Gemini uma informação utilizando a busca do Google como contexto

response = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é o próximo Google I/O?',
    config={"tools": [{"google_search": {}}]}
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {response.text}"))

Resposta:
 O próximo Google I/O será nos dias 20 e 21 de maio de 2025. O evento será transmitido ao vivo de Mountain View e online. Você pode se registrar para receber atualizações importantes sobre o evento.


In [ ]:
# Exibe a busca
print(f"Busca realizada: {response.candidates[0].grounding_metadata.web_search_queries}")
# Exibe as URLs nas quais ele se baseou
print(f"Páginas utilizadas na resposta: {', '.join([site.web.title for site in response.candidates[0].grounding_metadata.grounding_chunks])}")
print()
display(HTML(response.candidates[0].grounding_metadata.search_entry_point.rendered_content))

Busca realizada: ['Próximo evento Google I/O', 'data do próximo Google I/O']
Páginas utilizadas na resposta: google.com, googleblog.com, blog.google, io.google



In [ ]:
# Instalar Framework ADK de agentes do Google ################################################
!pip install -q google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.

In [ ]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [ ]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [ ]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
##########################################
# --- Agente 1: Coletor de Necessidades --- #
##########################################

def agente_coletor(needs, budget):
    coletor = Agent(
        name="agente_coletor",
        model="gemini-2.0-flash",
        description="Agente que coleta informações sobre necessidades e orçamento do usuário.",
        tools=[],
        instruction="""
            Você é um especialista em hardware de computadores. Sua tarefa é obter as necessidades detalhadas do usuário
            (como uso para jogos, trabalho, edição de vídeo, etc.) e seu orçamento total disponível para montagem de um PC.
            Baseado nessas informações, descreva os requisitos principais para as peças do computador (como CPU, GPU, RAM, etc.).
        """
    )
    entrada_do_agente_coletor = f"Necessidades do usuário: {needs}\nOrçamento: {budget}"
    requisitos = call_agent(coletor, entrada_do_agente_coletor)
    return requisitos

In [ ]:
###########################################
# --- Agente 2: Pesquisador de Peças --- #
###########################################

def agente_pesquisador(requisitos):
    pesquisador = Agent(
        name="agente_pesquisador",
        model="gemini-2.0-flash",
        description="Agente que pesquisa componentes de hardware disponíveis no mercado.",
        tools=[google_search],
        instruction="""
            Você é um pesquisador de hardware especializado em encontrar as melhores opções de peças de PC.
            Use a ferramenta do Google (google_search) para buscar componentes que atendam aos requisitos fornecidos.
            Identifique até 3 opções de cada peça necessária (CPU, GPU, RAM, etc.) dentro do orçamento estimado.
            Retorne as informações detalhadas, incluindo preços, marcas e especificações.
        """
    )
    entrada_do_agente_pesquisador = f"Requisitos: {requisitos}"
    componentes = call_agent(pesquisador, entrada_do_agente_pesquisador)
    return componentes

In [ ]:
#############################################
# --- Agente 3: Planejador do Build --- #
#############################################

def agente_planejador(componentes):
    planejador = Agent(
        name="agente_planejador",
        model="gemini-2.0-flash",
        description="Agente que planeja o build completo com base nas peças pesquisadas.",
        tools=[],
        instruction="""
            Você é um planejador de configuração de computadores. Com base nos componentes pesquisados, escolha
            as combinações mais compatíveis e otimizadas para montar o PC.
            Garanta que todas as peças sejam compatíveis (como CPU e placa-mãe) e estejam dentro do orçamento.
            Retorne um plano detalhado do build, com as peças selecionadas e justificativas para cada escolha.
        """
    )
    entrada_do_agente_planejador = f"Componentes pesquisados: {componentes}"
    build = call_agent(planejador, entrada_do_agente_planejador)
    return build

In [ ]:
##############################################
# --- Agente 4: Validador Técnico --- #
##############################################

def agente_validador(build):
    validador = Agent(
        name="agente_validador",
        model="gemini-2.0-flash",
        description="Agente que valida a compatibilidade técnica do build completo.",
        tools=[],
        instruction="""
            Você é um especialista técnico em hardware. Sua tarefa é revisar o build fornecido, verificando
            a compatibilidade entre as peças (como socket da CPU e placa-mãe, potência da fonte, etc.).
            Indique se o build está correto ou se precisa de ajustes. Caso precise de ajustes, sugira substituições adequadas.
        """
    )
    entrada_do_agente_validador = f"Build fornecido: {build}"
    validacao = call_agent(validador, entrada_do_agente_validador)
    return validacao

In [ ]:
###############################################
# --- Agente 5: Otimizador de Preços --- #
###############################################

def agente_otimizador(build_planejado):
    otimizador = Agent(
        name="agente_otimizador",
        model="gemini-2.0-flash",
        instruction="""
        Você é um assistente de compras que ajuda a encontrar os melhores preços para componentes de PCs.
        Receba a lista de componentes validados e busque os menores preços em lojas confiáveis e brasileiras.
        Retorne uma tabela com o preço, loja e link para cada componente, além do custo total em reais.
        """,
        description="Agente que otimiza os preços dos componentes.",
        tools=[google_search]
    )
    entrada_do_agente_otimizador = f"Build planejado: {build_planejado}"
    precos = call_agent(otimizador, entrada_do_agente_otimizador)
    return precos

In [ ]:
###############################################
# --- Agente 6: Simulador de Perfomance --- #
###############################################

def agente_simulador(build_planejado):
    simulador = Agent(
        name="agente_simulador",
        model="gemini-2.0-flash",
        instruction="""
        Você é um simulador de performance que avalia o desempenho de configurações de PCs.
        Receba os componentes e simule o desempenho para cenários como jogos, edição de vídeo e multitarefa.
        Retorne estimativas de FPS em jogos, tempos de renderização e eficiência multitarefa.
        """,
        description="Agente que simula o desempenho da configuração planejada.",
        tools=[]
    )
    entrada_do_agente_simulador = f"Build planejado: {build_planejado}"
    desempenho = call_agent(simulador, entrada_do_agente_simulador)
    return desempenho

In [ ]:
###############################################
# --- Agente 7: Revisor e Sugestões Extras --- #
###############################################

def agente_revisor(build):
    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.0-flash",
        description="Agente que revisa o build final e sugere melhorias adicionais.",
        tools=[],
        instruction="""
            Você é um revisor especializado em configurações de PCs. Sua tarefa é avaliar o build final validado e sugerir
            melhorias extras ou acessórios opcionais que possam otimizar o desempenho ou melhorar a experiência do usuário.
            Exemplos incluem upgrades de armazenamento, sistemas de refrigeração melhores ou periféricos complementares.
            Certifique-se de que as sugestões sejam úteis e viáveis dentro do orçamento fornecido.
        """
    )
    entrada_do_agente_revisor = f"Build validado: {build}"
    sugestoes = call_agent(revisor, entrada_do_agente_revisor)
    return sugestoes

In [ ]:
data_de_hoje = date.today().strftime("%d/%m/%Y")

print("🚀 Iniciando o Sistema de Assistência para Montagem de PCs com Múltiplos Agentes 🚀")

# --- Obter Informações do Usuário ---
needs = input("❓ Por favor, descreva suas necessidades (exemplo: jogos, edição de vídeo, trabalho, etc.): ")
budget = input("💰 Qual é o seu orçamento disponível para o PC? (em reais): ")

# Inserir lógica do sistema de agentes ################################################
if not needs or not budget:
    print("Você esqueceu de fornecer todas as informações necessárias!")
else:
    print(f"Maravilha! Vamos planejar a configuração ideal para suas necessidades dentro do orçamento de R$ {budget}.")

    requisitos = agente_coletor(needs, budget)
    print("\n--- 📝 Resultado do Agente 1 (Coletor de Necessidades) ---\n")
    display(to_markdown(requisitos))
    print("--------------------------------------------------------------")

    componentes_pesquisados = agente_pesquisador(requisitos)
    print("\n--- 📝 Resultado do Agente 2 (Pesquisador) ---\n")
    display(to_markdown(componentes_pesquisados))
    print("--------------------------------------------------------------")

    build_planejado = agente_planejador(componentes_pesquisados)
    print("\n--- 📝 Resultado do Agente 3 (Planejador) ---\n")
    display(to_markdown(build_planejado))
    print("--------------------------------------------------------------")

    validacao_final = agente_validador(build_planejado)
    print("\n--- 📝 Resultado do Agente 4 (Validador Técnico) ---\n")
    display(to_markdown(validacao_final))
    print("--------------------------------------------------------------")

    otimizacao_precos = agente_otimizador(validacao_final)
    print("\n--- 📝 Resultado do Agente 5 (Otimizador de Preços) ---\n")
    display(to_markdown(otimizacao_precos))
    print("--------------------------------------------------------------")

    desempenho_simulado = agente_simulador(validacao_final)
    print("\n--- 📝 Resultado do Agente 6 (Simulador de Performance) ---\n")
    display(to_markdown(desempenho_simulado))
    print("--------------------------------------------------------------")

    sugestoes_extras = agente_revisor(validacao_final)
    print("\n--- 📝 Resultado do Agente 7 (Revisor e Sugestões Extras) ---\n")
    display(to_markdown(sugestoes_extras))
    print("--------------------------------------------------------------")


🚀 Iniciando o Sistema de Assistência para Montagem de PCs com Múltiplos Agentes 🚀
❓ Por favor, descreva suas necessidades (exemplo: jogos, edição de vídeo, trabalho, etc.): Jogos
💰 Qual é o seu orçamento disponível para o PC? (em reais): 4.000
Maravilha! Vamos planejar a configuração ideal para suas necessidades dentro do orçamento de R$ 4.000.

--- 📝 Resultado do Agente 1 (Coletor de Necessidades) ---



> Ok, com um orçamento de R$ 4.000 para um PC gamer, vamos focar nos componentes mais importantes para ter um bom desempenho nos jogos. Aqui estão os requisitos principais que devemos considerar:
> 
> *   **Processador (CPU):** Um bom ponto de partida seria um AMD Ryzen 5 5600 ou um Intel Core i5-12400F. Ambos oferecem um excelente desempenho em jogos e estão dentro de uma faixa de preço acessível.
> 
> *   **Placa de Vídeo (GPU):** A placa de vídeo é o componente mais crucial para jogos. Com esse orçamento, podemos mirar em uma AMD Radeon RX 6600 ou uma NVIDIA GeForce RTX 3050. Se possível, tentar encontrar uma RX 6600 XT usada pode ser uma ótima opção para ter ainda mais desempenho.
> 
> *   **Memória RAM:** 16GB de RAM DDR4 com uma velocidade de pelo menos 3200MHz é o ideal para jogos modernos.
> 
> *   **Armazenamento:** Um SSD NVMe de 500GB ou 1TB é essencial para tempos de carregamento rápidos nos jogos e no sistema operacional. Se o orçamento permitir, adicionar um HD de 1TB para armazenar mais jogos e arquivos pode ser interessante.
> 
> *   **Placa-mãe:** Uma placa-mãe que seja compatível com o processador escolhido e que tenha recursos suficientes para suas necessidades (slots de expansão, conexões, etc.).
> 
> *   **Fonte de Alimentação:** Uma fonte de 550W ou 650W de boa qualidade com certificação 80+ Bronze deve ser suficiente para alimentar o sistema com folga.
> 
> *   **Gabinete:** Escolha um gabinete que tenha boa ventilação para manter os componentes resfriados e que tenha espaço suficiente para todos os componentes.
> 
> *   **Cooler:** Um air cooler é suficiente para resfriar o processador. Se possível, invista em um cooler um pouco melhor para garantir temperaturas mais baixas e menor ruído.
> 
> Esses são os requisitos principais para montar um PC gamer com um orçamento de R$ 4.000. É importante pesquisar os preços e comparar diferentes marcas e modelos para encontrar as melhores ofertas e garantir que todos os componentes sejam compatíveis entre si.


--------------------------------------------------------------

--- 📝 Resultado do Agente 2 (Pesquisador) ---



> Ok, entendi os requisitos para o PC gamer com orçamento de R$ 4.000. Vou pesquisar os componentes e apresentar algumas opções que se encaixam nas suas necessidades.
> 
> 
> Com base na sua solicitação, aqui estão algumas opções de componentes para montar um PC gamer com um orçamento de R$ 4.000:
> 
> **Processador (CPU):**
> 
> *   **AMD Ryzen 5 5600:** Este processador oferece um excelente desempenho em jogos e possui um bom custo-benefício. O preço pode variar, mas geralmente está na faixa de R$ 800 a R$ 900.
> *   **Intel Core i5-12400F:** Similar ao Ryzen 5 5600 em termos de desempenho, este processador Intel também é uma ótima opção. Você pode encontrá-lo por cerca de R$ 700 a R$ 800.
> 
> **Placa de Vídeo (GPU):**
> 
> *   **AMD Radeon RX 6600:** Esta placa de vídeo é uma excelente escolha para jogos em 1080p e oferece um bom desempenho por um preço acessível. O preço geralmente fica entre R$ 1.200 e R$ 1.500.
> *   **NVIDIA GeForce RTX 3050:** Similar em desempenho à RX 6600, a RTX 3050 também é uma boa opção para jogos em 1080p. O preço pode variar entre R$ 1.300 e R$ 1.600.
> *   **AMD Radeon RX 6600 XT (Usada):** Se você conseguir encontrar uma RX 6600 XT usada em bom estado, pode ser uma ótima opção para obter ainda mais desempenho dentro do seu orçamento. Os preços de modelos usados podem variar, mas procure por ofertas entre R$ 1.000 e R$ 1.300.
> 
> **Memória RAM:**
> 
> *   **16GB DDR4 3200MHz:** Essencial para jogos modernos, procure por kits de 16GB (2x8GB) com velocidade de 3200MHz. Algumas opções incluem Corsair Vengeance LPX, Kingston Fury Beast e Team Group T-Create. O preço deve ficar entre R$ 250 e R$ 350.
> 
> **Armazenamento:**
> 
> *   **SSD NVMe 500GB/1TB:** Um SSD NVMe é crucial para tempos de carregamento rápidos. Considere opções como Kingston NV3, Crucial P3 Plus ou Adata Legend 710.
>     *   500GB: R$ 200 - R$ 300.
>     *   1TB: R$ 300 - R$ 450.
> *   **HD 1TB (Opcional):** Se o orçamento permitir, um HD de 1TB pode ser adicionado para armazenamento adicional de jogos e arquivos. Opções como Western Digital Blue ou Seagate Barracuda podem ser encontradas por cerca de R$ 200 a R$ 300.
> 
> **Placa-Mãe:**
> 
> *   **Para AMD Ryzen 5 5600:** Uma placa-mãe com chipset B450, B550 ou A520 é compatível.
> *   **Para Intel Core i5-12400F:** Uma placa-mãe com chipset H610, B660 ou H670 é necessária.
> *   O preço de uma placa-mãe deve variar entre R$ 400 e R$ 700, dependendo do chipset e recursos.
> 
> **Fonte de Alimentação:**
> 
> *   **550W/650W 80+ Bronze:** Uma fonte de alimentação de boa qualidade com certificação 80+ Bronze é essencial para garantir a estabilidade do sistema. Algumas opções incluem Corsair CV550, MSI MAG A650BN e Gamemax GP650. O preço deve variar entre R$ 250 e R$ 400.
> 
> **Gabinete:**
> 
> *   **Com bom fluxo de ar:** Escolha um gabinete com boa ventilação para manter os componentes resfriados. Opções como Corsair, Cooler Master e Redragon oferecem bons gabinetes com foco em fluxo de ar. O preço deve variar entre R$ 200 e R$ 400.
> 
> **Cooler:**
> 
> *   **Air Cooler:** Um air cooler é suficiente para resfriar o processador. Opções como Cooler Master Hyper 212 ou Noctua NH-U12S são boas escolhas. O preço deve variar entre R$ 150 e R$ 300.
> 
> Lembre-se de pesquisar os preços em diferentes lojas e comparar as opções disponíveis para encontrar as melhores ofertas. Além disso, verifique a compatibilidade de todos os componentes antes de finalizar a compra.
> 


--------------------------------------------------------------

--- 📝 Resultado do Agente 3 (Planejador) ---



...

--------------------------------------------------------------

--- 📝 Resultado do Agente 6 (Simulador de Performance) ---



> Ok, build ajustado e compatível! Vamos simular o desempenho dessa máquina para te dar uma ideia do que esperar:
> 
> **Desempenho Estimado:**
> 
> *   **Jogos (1080p - configurações altas/ultra):**
>     *   **Títulos AAA (Cyberpunk 2077, Red Dead Redemption 2):** 55-70 FPS (com ajustes nas configurações, pode chegar a 60 FPS estáveis)
>     *   **Jogos Competitivos (CS:GO, Valorant, Rainbow Six Siege):** 200+ FPS
>     *   **Jogos Mais Leves (Fortnite, Overwatch 2):** 140+ FPS
> *   **Edição de Vídeo (1080p/4K):**
>     *   **Renderização de Vídeos Curtos (até 10 minutos):** Rápida e eficiente, com tempos de renderização razoáveis.
>     *   **Renderização de Vídeos Longos (20+ minutos):**  Tempo de renderização dependerá da complexidade do projeto, mas será significativamente mais rápido do que em builds mais básicos.
> *   **Multitarefa:**
>     *   **Navegação na Web, Edição de Documentos, Streaming de Música:** Desempenho excelente e sem gargalos.
>     *   **Jogos + Streaming:**  Capaz de realizar streaming em 1080p com boa qualidade, mas pode haver uma pequena queda no FPS dependendo do jogo.
>     *   **Edição de Vídeo + Outras Tarefas:** O Ryzen 5 5600 conseguirá lidar bem com multitarefas exigentes, mas o tempo de renderização pode ser um pouco afetado.
> 
> **Considerações:**
> 
> *   **RX 6600:** Uma excelente placa para 1080p, mas pode exigir alguns ajustes nas configurações para rodar jogos mais pesados em resoluções mais altas (1440p).
> *   **16GB RAM:** Suficiente para a maioria dos jogos e tarefas, mas para edição de vídeo mais pesada, considerar 32GB RAM no futuro pode ser interessante.
> *   **SSD NVMe:** Essencial para tempos de carregamento rápidos e responsividade do sistema.
> *   **Air Cooler:** Um bom air cooler é importante para manter as temperaturas sob controle, especialmente durante sessões de jogos prolongadas ou renderização de vídeo.
> *   **B550M:** A escolha da B550M foi crucial para garantir a compatibilidade e aproveitar os benefícios do PCIe 4.0.
> 
> **Em resumo:**  Este build oferece um ótimo equilíbrio entre preço e desempenho, ideal para jogos em 1080p, edição de vídeo e multitarefa. A troca da placa-mãe foi uma excelente decisão para garantir a compatibilidade e evitar dores de cabeça com atualizações de BIOS.
> 


--------------------------------------------------------------

--- 📝 Resultado do Agente 7 (Revisor e Sugestões Extras) ---



> O build está excelente! A troca da placa-mãe B450M pela B550M foi uma escolha inteligente para garantir a compatibilidade e aproveitar os recursos mais recentes.
> 
> **Sugestões de Melhorias Adicionais e Acessórios Opcionais:**
> 
> 1.  **Armazenamento:**
>     *   **Adicionar um HD de 1TB** (R$ 200 - R$ 300): Embora o SSD NVMe de 500GB seja ótimo para o sistema operacional e jogos mais usados, um HD adicional oferece espaço para armazenar uma biblioteca maior de jogos, vídeos, fotos e outros arquivos grandes. Isso evita a necessidade de desinstalar jogos para liberar espaço no SSD.
>     *   **Upgrade do SSD para 1TB** (R$ 400 - R$ 500): Se o orçamento permitir, aumentar a capacidade do SSD NVMe para 1TB é uma excelente opção. Isso proporciona mais espaço para instalar jogos e aplicativos, resultando em tempos de carregamento mais rápidos e uma experiência geral mais fluida.
> 
> 2.  **Refrigeração:**
>     *   **Upgrade do Air Cooler para um modelo mais robusto** (R$ 300 - R$ 400): Se você planeja fazer overclock no processador ou mora em um local quente, investir em um air cooler de alta performance pode ajudar a manter as temperaturas sob controle e garantir um desempenho mais estável a longo prazo. Marcas como Noctua, be quiet! e Cooler Master oferecem excelentes opções.
> 
> 3.  **Periféricos:**
>     *   **Teclado Mecânico** (R$ 200 - R$ 500): Um teclado mecânico oferece uma experiência de digitação mais precisa e responsiva, o que pode ser especialmente útil em jogos. Há uma grande variedade de opções disponíveis, com diferentes tipos de switches e recursos.
>     *   **Mouse Gamer** (R$ 100 - R$ 300): Um mouse gamer com DPI ajustável e botões programáveis pode melhorar significativamente sua jogabilidade. Procure por um modelo que se encaixe bem na sua mão e ofereça os recursos que você precisa.
>     *   **Headset Gamer** (R$ 200 - R$ 400): Um headset gamer de qualidade oferece áudio imersivo e comunicação clara com outros jogadores. Modelos com som surround virtual podem te dar uma vantagem competitiva em jogos.
> 
> 4.  **Estabilizador/Filtro de Linha:**
>     *   **Estabilizador/Filtro de Linha** (R$ 100 - R$ 200): Para proteger o seu investimento contra picos de tensão e ruídos na rede elétrica, um estabilizador ou filtro de linha é altamente recomendado.
> 
> **Observações:**
> 
> *   As sugestões acima são opcionais e dependem do seu orçamento e necessidades.
> *   Priorize as melhorias que trarão o maior impacto na sua experiência de uso.
> *   Pesquise e compare preços antes de fazer qualquer compra.
> 
> Com essas adições, seu PC gamer ficará ainda mais completo e preparado para te proporcionar horas de diversão!
> 


--------------------------------------------------------------
